In [1]:
from google.colab import drive
drive.mount('/content/drive')
# import os
# os.makedirs('/content/drive/MyDrive/sentiment_project', exist_ok=True)


Mounted at /content/drive


In [2]:
from IPython import get_ipython
from IPython.display import display

In [3]:
import json
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
import string
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
import logging
from tqdm import tqdm
import torch

In [ ]:
# Load the dataset  | we will start with 500000
def load_json_lines_to_df(filepath: str, max_lines=500000):
    data = []
    with open(filepath, 'r') as file:
        for i, line in enumerate(file):
            if max_lines is not None and i >= max_lines:
                break
            try:
                data.append(json.loads(line.strip()))
            except json.JSONDecodeError as e:
                print(f"Error decoding line {i}: {e}")
    return pd.DataFrame(data)

In [ ]:
# Get the sentiment from the rating ( rating-based sentiment analysis)
def get_sentiment(score):
    if score >= 4:
        return 'positive'
    elif score == 3:
        return 'neutral'
    else:
        return 'negative'

In [ ]:
# Clean : lower case only, no punctuation, no numbers, no multiple space
def clean_text(text):
    if not isinstance(text, str) or text.strip() == '':
        return None
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [ ]:
df=load_json_lines_to_df("/content/drive/MyDrive/sentiment_project/Software.jsonl")

In [ ]:
df.info()
df.describe()
df.shape
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 10 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   rating             500000 non-null  float64
 1   title              500000 non-null  object 
 2   text               500000 non-null  object 
 3   images             500000 non-null  object 
 4   asin               500000 non-null  object 
 5   parent_asin        500000 non-null  object 
 6   user_id            500000 non-null  object 
 7   timestamp          500000 non-null  int64  
 8   helpful_vote       500000 non-null  int64  
 9   verified_purchase  500000 non-null  bool   
dtypes: bool(1), float64(1), int64(2), object(6)
memory usage: 34.8+ MB


,rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase
0,1.0,malware,mcaffee IS malware,[],B07BFS3G7P,B0BQSK9QCF,AGCI7FAH4GL5FI65HYLKWTMFZ2CQ,1562182632076,0,False
1,5.0,Lots of Fun,I love playing tapped out because it is fun to...,[],B00CTQ6SIG,B00CTQ6SIG,AHSPLDNW5OOUK2PLH7GXLACFBZNQ,1424120336000,0,True
2,5.0,Light Up The Dark,I love this flashlight app! It really illumin...,[],B0066WJLU6,B0066WJLU6,AHSPLDNW5OOUK2PLH7GXLACFBZNQ,1362399267000,0,True
3,4.0,Fun game,One of my favorite games,[],B00KCYMAWK,B00KCYMAWK,AH6CATODIVPVUOJEWHRSRCSKAOHA,1561061428662,0,True
4,4.0,I am not that good at it but my kids are,Cute game. I am not that good at it but my kid...,[],B00P1RK566,B00P1RK566,AEINY4XOINMMJCK5GZ3M6MMHBN6A,1418257196000,0,True


In [ ]:
df.isna().sum()

,0
rating,0
title,0
text,0
images,0
asin,0
parent_asin,0
user_id,0
timestamp,0
helpful_vote,0
verified_purchase,0


In [ ]:
df.describe()

,rating,timestamp,helpful_vote
count,2.000000e+06,2.000000e+06,2.000000e+06
mean,3.907688e+00,1.483010e+12,4.599215e+00
std,1.461368e+00,8.945162e+10,3.032790e+01
min,1.000000e+00,9.433309e+11,0.000000e+00
25%,3.000000e+00,1.419959e+12,0.000000e+00
50%,5.000000e+00,1.473689e+12,0.000000e+00
75%,5.000000e+00,1.552263e+12,2.000000e+00
max,5.000000e+00,1.681509e+12,6.178000e+03


In [ ]:
df['label'] = df['rating'].apply(get_sentiment)

In [ ]:
df['verified_purchase'] = df['verified_purchase'].astype(int)

In [ ]:
df['full_text'] = (df['title'].fillna('') + ' ' + df['text'].fillna('')).str.strip()

In [ ]:
df['full_text'] = df['full_text'].apply(clean_text)

In [ ]:
df = df[df['full_text'].notna()]
df = df[df['full_text'].str.strip() != '']

In [ ]:
df = df.drop(columns=['asin', 'parent_asin', 'user_id', 'images'])

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_encoded = le.fit_transform(df['label'])

In [ ]:
le.classes_  # negative : 0, neutral : 1, positive : 2

array(['negative', 'neutral', 'positive'], dtype=object)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 499867 entries, 0 to 499999
Data columns (total 8 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   rating             499867 non-null  float64
 1   title              499867 non-null  object 
 2   text               499867 non-null  object 
 3   timestamp          499867 non-null  int64  
 4   helpful_vote       499867 non-null  int64  
 5   verified_purchase  499867 non-null  int64  
 6   label              499867 non-null  object 
 7   full_text          499867 non-null  object 
dtypes: float64(1), int64(3), object(4)
memory usage: 34.3+ MB


In [ ]:
# Suppress warnings
logging.getLogger("transformers").setLevel(logging.ERROR)

# Check for GPU
device = 0 if torch.cuda.is_available() else -1
print("Using device:", "GPU" if device == 0 else "CPU")

# Load model and tokenizer
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Create classifier pipeline
classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer, device=device, truncation=True)

# Prepare text
texts = df['full_text'].astype(str).tolist()
batch_size = 64  # adjust based on GPU RAM
predictions = []

for i in tqdm(range(0, len(texts), batch_size)):
    batch = texts[i:i+batch_size]
    try:
        results = classifier(batch, truncation=True)
        labels = [r['label'] for r in results]
        predictions.extend(labels)
    except Exception as e:
        print(f"Error at batch {i}: {e}")
        predictions.extend([None] * len(batch))

Using device: GPU


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

100%|██████████| 15622/15622 [1:23:35<00:00,  3.11it/s]


In [ ]:
df['sentiment_bert'] = predictions

In [ ]:
df.head(1000000)

,rating,title,text,timestamp,helpful_vote,verified_purchase,label,full_text,sentiment_bert
0,1.0,malware,mcaffee IS malware,1562182632076,0,0,negative,malware mcaffee is malware,NEGATIVE
1,5.0,Lots of Fun,I love playing tapped out because it is fun to...,1424120336000,0,1,positive,lots of fun i love playing tapped out because ...,POSITIVE
2,5.0,Light Up The Dark,I love this flashlight app! It really illumin...,1362399267000,0,1,positive,light up the dark i love this flashlight app i...,POSITIVE
3,4.0,Fun game,One of my favorite games,1561061428662,0,1,positive,fun game one of my favorite games,POSITIVE
4,4.0,I am not that good at it but my kids are,Cute game. I am not that good at it but my kid...,1418257196000,0,1,positive,i am not that good at it but my kids are cute ...,POSITIVE
...,...,...,...,...,...,...,...,...,...
999995,5.0,Amazing,"This game is so fun, I love it.",1434671221000,0,1,positive,amazing this game is so fun i love it,POSITIVE
999996,1.0,terrible in so many ways! :(,"My little sister got this game, so I decided t...",1420216104000,17,1,negative,terrible in so many ways my little sister got ...,NEGATIVE
999997,1.0,glitches your worlds,"This app is pretty great, but sometimes it wil...",1420127748000,5,1,negative,glitches your worlds this app is pretty great ...,NEGATIVE
999998,2.0,great! unless you don't have Facebook,"So, I had a preteen and a toddler. I was on l...",1418903383000,0,1,negative,great unless you dont have facebook so i had a...,NEGATIVE


In [ ]:
# Normalize your original labels to lowercase for consistency
df['label'] = df['label'].str.lower()

# Normalize BERT labels to match your original format
df['sentiment_bert'] = df['sentiment_bert'].str.lower()

In [ ]:
df['label_match'] = df['label'] == df['sentiment_bert']

In [ ]:
# View mismatches
mismatches = df[df['label_match'] == False]
print(f"Number of mismatches: {len(mismatches)}")

pd.set_option('display.max_colwidth', None)  # So full text is shown
mismatches[['full_text', 'label', 'sentiment_bert']]

Number of mismatches: 228164


,full_text,label,sentiment_bert
7,great antivirus product not sure what else can be said about norton products i have used them for years on all my desk and laptop computers easy to install and protects my harddrives well not sure i see what one of the other reviewers stated about upsells from norton havent seen them yet even after almost weeks of daily use anyway if you are looking for antivirus software that can be easily installed on three of your devices this is for you,positive,negative
11,great product i had a trial version of microsoft office which came with my laptop so i was in the market for the full blown version that would work with my vista system this product was easy to use and install and walked me throught the entire process i would highly recommend to anyone,positive,negative
14,mobileme upgrade again cheaper to buy the amazon upgrade versus paining apples high pricebr to be honest i wonder at times why i do this every year the features really arent any better each year i dont have a lot backed up but do have somebr i havent hung a website up so whybr keeps an email available to me my wife and daughter to communicate with yes i have gmail and hotmail but both have been spammed and taken over even w password changesbr so i pay the fee and upgrade thanks to amazon,positive,negative
19,silly penguin lot of fun but can be very frustrating only because im a moron times while playing the gamemade me laugh myself and try harderjust a fun simple game,positive,negative
24,reef rescue i like this game ok but wish there were less that you must purchase eh ill play until i get tired of the pay for this pay for that eh wish it could be the good old days and you could play a game for fun,positive,negative
...,...,...,...
999981,fun but too many ads i really enjoy this game but find the frequent ads extremely annoying i now get an ad after about every third game i also own this on another tablet and i dont get any ads on there a game i purchased should not interrupt play so often with unwanted and intrusive ads i may have to stop playing this game on this device,neutral,negative
999982,not a real youtube app i don’t think this is an actual youtube app it feels like the youtube website cramped into a app application it docent have a lot of the youtube app features like brining down the current video to brows for more videos you have to literally tap the back button it feels like a desktop version of the website honestly,neutral,negative
999985,u s a has some things that i would read,neutral,positive
999986,could be better this game is slow to load on my kindle it constantly freezes and reloads when you buy items through amazon wild card for example it reloads causing you to loss a game also the wild card doesnt always appear,neutral,negative


In [ ]:
df['label'] = df['label'].str.lower()
df['sentiment_bert'] = df['sentiment_bert'].str.lower()

df_filtered = df[df['label'] != 'neutral']

df_filtered['label_match'] = df_filtered['label'] == df_filtered['sentiment_bert']

mismatches = df_filtered[df_filtered['label_match'] == False]
print(f"Number of mismatches (excluding neutral): {len(mismatches)}")


pd.set_option('display.max_colwidth', None)
mismatches[['full_text', 'label', 'sentiment_bert']]


Number of mismatches (excluding neutral): 127137


<ipython-input-39-ce2105732005>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['label_match'] = df_filtered['label'] == df_filtered['sentiment_bert']


,full_text,label,sentiment_bert
7,great antivirus product not sure what else can be said about norton products i have used them for years on all my desk and laptop computers easy to install and protects my harddrives well not sure i see what one of the other reviewers stated about upsells from norton havent seen them yet even after almost weeks of daily use anyway if you are looking for antivirus software that can be easily installed on three of your devices this is for you,positive,negative
11,great product i had a trial version of microsoft office which came with my laptop so i was in the market for the full blown version that would work with my vista system this product was easy to use and install and walked me throught the entire process i would highly recommend to anyone,positive,negative
14,mobileme upgrade again cheaper to buy the amazon upgrade versus paining apples high pricebr to be honest i wonder at times why i do this every year the features really arent any better each year i dont have a lot backed up but do have somebr i havent hung a website up so whybr keeps an email available to me my wife and daughter to communicate with yes i have gmail and hotmail but both have been spammed and taken over even w password changesbr so i pay the fee and upgrade thanks to amazon,positive,negative
19,silly penguin lot of fun but can be very frustrating only because im a moron times while playing the gamemade me laugh myself and try harderjust a fun simple game,positive,negative
24,reef rescue i like this game ok but wish there were less that you must purchase eh ill play until i get tired of the pay for this pay for that eh wish it could be the good old days and you could play a game for fun,positive,negative
...,...,...,...
999935,nuts that is anyone who doesnt subscribe to acorn,positive,negative
999948,quicken helps i have used quicken for years now i am not sure what i would do without it,positive,negative
999958,great software that simplifies the tax return process this is probably the th or th year that ive been using turbotax and as usual it simplifies my tax return process i just make sure to save the documents i need and transcribe what i need to it saves even more time if youve used turbotax in the past because it can import data from the past year and help minimize data entry so if you work for the same company then that employer info is already there for you in the w section i love that i can either skip sections or have it walk me through everything to make sure i take advantage of all the tax advantages i can,positive,negative
999968,i really like this program wow i really like this program i have rosetta stone instant immersion and pimsleur yup all of them in french rosetta stone is weird i cant get past the first lesson or two because i cant just memorize a phrase with a picture i want to know what each word is for and why it is used pimsleur is better for me but at some level usually around lesson we are using forms of verbs and it isnt clear why we use au restaurant and a lhotel to mean at the xbr br fluenz actually explains these things that bug me best each lesson has about a minute lecture that breaks down the grammar so that you understand what each term means and what the discrepancies are from french to english then you can spend as much time as you need on activities that teach reading writing and listening comprehensionbr br im through lesson and have learned a lot of the issues that were impeding my completion using the other two methods im very glad i bought one more language program and feel like i am having a lot of success with this process thanks to the other reviewers also for helping me make this decision,positive,negative


*nlptown/bert-base-multilingual-uncased-sentiment : Review star predictor*

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

texts = df['full_text'].astype(str).tolist()
predictions = []

batch_size = 32
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for i in tqdm(range(0, len(texts), batch_size)):
    batch = texts[i:i+batch_size]
    encoding = tokenizer(batch, return_tensors='pt', padding=True, truncation=True, max_length=512)
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=1).cpu().numpy()

    for star in preds:
        if star <= 1:
            predictions.append("negative")
        elif star == 2:
            predictions.append("neutral")
        else:
            predictions.append("positive")

df['sentiment_bert_2'] = predictions


 65%|██████▌   | 20364/31244 [1:54:27<44:54,  4.04it/s]

In [ ]:
df_small = df.head(50000)

df_small = df_small.reset_index(drop=True)

In [ ]:
!pip install -U google-generativeai

In [ ]:
import google.generativeai as genai
import pandas as pd
from tqdm import tqdm
from google.colab import userdata
key=userdata.get('gemini_key')
genai.configure(api_key=key)

In [ ]:
model = genai.GenerativeModel("gemini-pro")

In [ ]:
def classify_sentiment(text, api_key=key):
    client = genai.Client(api_key=api_key)

    prompt = f"""Classify the sentiment of the following text as Positive, Neutral, or Negative:

    Text: "{text}"

    Sentiment:"""

    try:
        response = client.models.generate_content(
            model="gemini-1.5-flash",
            contents=prompt
        )
        sentiment = response.text.strip().lower()

        if "positive" in sentiment:
            return "positive"
        elif "negative" in sentiment:
            return "negative"
        elif "neutral" in sentiment:
            return "neutral"
        else:
            return "unknown"

    except Exception as e:
        print(f"Error processing text: {e}")
        return "error"

In [ ]:
from google import genai
client=genai.Client(api_key=key)
response = client.models.generate_content(
    model="gemini-1.5-pro", contents="Explain how AI works in a few words"
)
print(response.text)

ClientError: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_input_token_count', 'quotaId': 'GenerateContentInputTokensPerModelPerMinute-FreeTier', 'quotaDimensions': {'location': 'global', 'model': 'gemini-1.5-pro'}}, {'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_requests', 'quotaId': 'GenerateRequestsPerMinutePerProjectPerModel-FreeTier', 'quotaDimensions': {'location': 'global', 'model': 'gemini-1.5-pro'}}, {'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_requests', 'quotaId': 'GenerateRequestsPerDayPerProjectPerModel-FreeTier', 'quotaDimensions': {

In [ ]:
tqdm.pandas()  # Add progress bar support

# Apply Gemini classification
df_small['sentiment_gemini'] = df_small['full_text'].progress_apply(classify_sentiment)

  0%|          | 19/50000 [00:07<5:31:17,  2.51it/s]

Error processing text: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_requests', 'quotaId': 'GenerateRequestsPerMinutePerProjectPerModel-FreeTier', 'quotaDimensions': {'location': 'global', 'model': 'gemini-1.5-flash'}, 'quotaValue': '15'}]}, {'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googleapis.com/google.rpc.RetryInfo', 'retryDelay': '42s'}]}}


  0%|          | 21/50000 [00:07<3:55:24,  3.54it/s]

Error processing text: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_requests', 'quotaId': 'GenerateRequestsPerMinutePerProjectPerModel-FreeTier', 'quotaDimensions': {'location': 'global', 'model': 'gemini-1.5-flash'}, 'quotaValue': '15'}]}, {'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googleapis.com/google.rpc.RetryInfo', 'retryDelay': '41s'}]}}
Error processing text: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, pleas

  0%|          | 23/50000 [00:08<2:46:18,  5.01it/s]

...

  1%|          | 422/50000 [01:21<1:41:31,  8.14it/s]

Error processing text: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_requests', 'quotaId': 'GenerateRequestsPerMinutePerProjectPerModel-FreeTier', 'quotaDimensions': {'location': 'global', 'model': 'gemini-1.5-flash'}, 'quotaValue': '15'}]}, {'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googleapis.com/google.rpc.RetryInfo', 'retryDelay': '28s'}]}}
Error processing text: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, pleas

  1%|          | 424/50000 [01:21<2:39:34,  5.18it/s]

Error processing text: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_requests', 'quotaId': 'GenerateRequestsPerMinutePerProjectPerModel-FreeTier', 'quotaDimensions': {'location': 'global', 'model': 'gemini-1.5-flash'}, 'quotaValue': '15'}]}, {'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googleapis.com/google.rpc.RetryInfo', 'retryDelay': '27s'}]}}
Error processing text: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, pleas

KeyboardInterrupt: 

In [ ]:
df_small

,rating,title,text,timestamp,helpful_vote,verified_purchase,label,full_text
0,1.0,malware,mcaffee IS malware,1562182632076,0,0,negative,malware mcaffee is malware
1,5.0,Lots of Fun,I love playing tapped out because it is fun to...,1424120336000,0,1,positive,lots of fun i love playing tapped out because ...
2,5.0,Light Up The Dark,I love this flashlight app! It really illumin...,1362399267000,0,1,positive,light up the dark i love this flashlight app i...
3,4.0,Fun game,One of my favorite games,1561061428662,0,1,positive,fun game one of my favorite games
4,4.0,I am not that good at it but my kids are,Cute game. I am not that good at it but my kid...,1418257196000,0,1,positive,i am not that good at it but my kids are cute ...
...,...,...,...,...,...,...,...,...
49995,1.0,Can I give this 0 Stars?,"Seriously, I feel crazy for getting so angry a...",1551842273495,1,1,negative,can i give this stars seriously i feel crazy f...
49996,5.0,Five Stars,Great gift cards for client. Everyone loves to...,1503635071176,0,1,positive,five stars great gift cards for client everyon...
49997,4.0,It was good,The games good but after I updated it keeps po...,1465940361000,94,1,positive,it was good the games good but after i updated...
49998,3.0,its okay,It's just a city with a little bit of fun,1418952383000,0,1,neutral,its okay its just a city with a little bit of fun


In [ ]:
origin=df.copy()

In [ ]:
df=origin.head(7000)

In [ ]:
df

,rating,title,text,timestamp,helpful_vote,verified_purchase,label,full_text
0,1.0,malware,mcaffee IS malware,1562182632076,0,0,negative,malware mcaffee is malware
1,5.0,Lots of Fun,I love playing tapped out because it is fun to...,1424120336000,0,1,positive,lots of fun i love playing tapped out because ...
2,5.0,Light Up The Dark,I love this flashlight app! It really illumin...,1362399267000,0,1,positive,light up the dark i love this flashlight app i...
3,4.0,Fun game,One of my favorite games,1561061428662,0,1,positive,fun game one of my favorite games
4,4.0,I am not that good at it but my kids are,Cute game. I am not that good at it but my kid...,1418257196000,0,1,positive,i am not that good at it but my kids are cute ...
...,...,...,...,...,...,...,...,...
6997,5.0,Always,I FREAKIN LOVE IT!!!,1570503162484,0,1,positive,always i freakin love it
6998,1.0,You have to pay for this,"It stinks, why should I pay for network tv tha...",1507262598769,1,1,negative,you have to pay for this it stinks why should ...
6999,5.0,and it has a simple and easy to use interface,It's played everything I've wanted it to witho...,1445647183000,3,1,positive,and it has a simple and easy to use interface ...
7000,5.0,Should come preloaded,This app has more features -- useful ones -- t...,1407399926000,0,1,positive,should come preloaded this app has more featur...


on 7000 first observations

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

device = torch.device("cpu")
model.to(device)
model.eval()

texts = df['full_text'].astype(str).tolist()
predictions = []

batch_size = 8

for i in tqdm(range(0, len(texts), batch_size)):
    batch = texts[i:i+batch_size]
    inputs = tokenizer(batch, padding=True, truncation=True, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=1).cpu().numpy()

    for star in preds:
        if star <= 1:
            predictions.append("negative")
        elif star == 2:
            predictions.append("neutral")
        else:
            predictions.append("positive")

df["sentiment"] = predictions

 15%|█▌        | 134/875 [04:35<25:25,  2.06s/it]


KeyboardInterrupt: 

In [ ]:
df

,rating,title,text,timestamp,helpful_vote,verified_purchase,label,full_text,sentiment
0,1.0,malware,mcaffee IS malware,1562182632076,0,0,negative,malware mcaffee is malware,negative
1,5.0,Lots of Fun,I love playing tapped out because it is fun to...,1424120336000,0,1,positive,lots of fun i love playing tapped out because ...,positive
2,5.0,Light Up The Dark,I love this flashlight app! It really illumin...,1362399267000,0,1,positive,light up the dark i love this flashlight app i...,positive
3,4.0,Fun game,One of my favorite games,1561061428662,0,1,positive,fun game one of my favorite games,positive
4,4.0,I am not that good at it but my kids are,Cute game. I am not that good at it but my kid...,1418257196000,0,1,positive,i am not that good at it but my kids are cute ...,positive
...,...,...,...,...,...,...,...,...,...
6997,5.0,Always,I FREAKIN LOVE IT!!!,1570503162484,0,1,positive,always i freakin love it,positive
6998,1.0,You have to pay for this,"It stinks, why should I pay for network tv tha...",1507262598769,1,1,negative,you have to pay for this it stinks why should ...,negative
6999,5.0,and it has a simple and easy to use interface,It's played everything I've wanted it to witho...,1445647183000,3,1,positive,and it has a simple and easy to use interface ...,positive
7000,5.0,Should come preloaded,This app has more features -- useful ones -- t...,1407399926000,0,1,positive,should come preloaded this app has more featur...,positive


In [ ]:
df.to_csv('/content/drive/MyDrive/sentiment_project/processed_reviews_7milles.csv', index=False)

on the next 7000

In [ ]:
df=origin[7000:14000]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

# Set device to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

texts = df['full_text'].astype(str).tolist()
predictions = []

batch_size = 8

for i in tqdm(range(0, len(texts), batch_size)):
    batch = texts[i:i+batch_size]

    inputs = tokenizer(batch, padding=True, truncation=True, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=1).cpu().numpy()

    for star in preds:
        if star <= 1:
            predictions.append("negative")
        elif star == 2:
            predictions.append("neutral")
        else:
            predictions.append("positive")

df["sentiment"] = predictions

100%|██████████| 4500/4500 [03:56<00:00, 19.01it/s]
<ipython-input-32-0b6504a43b64>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["sentiment"] = predictions


In [ ]:
df.to_csv('/content/drive/MyDrive/sentiment_project/processed_reviews_second_7milles.csv', index=False)

In [ ]:
df

,rating,title,text,timestamp,helpful_vote,verified_purchase,label,full_text,sentiment
7002,1.0,Not For Me.,Although I sudoku these left me totally confused.,1564316748359,0,1,negative,not for me although i sudoku these left me tot...,negative
7003,2.0,Fun for Some but Not for Me.,"This game did not appeal to me, I have already...",1526138904396,1,1,negative,fun for some but not for me this game did not ...,negative
7004,4.0,Love A&E,Just as good as I received as part of my satel...,1526138705757,0,1,positive,love ae just as good as i received as part of ...,positive
7005,3.0,I expected More!,"I expected more, that'll teach me.",1524056164247,0,1,neutral,i expected more i expected more thatll teach me,neutral
7006,3.0,Settled for this game but it didnt meet my exp...,I was looking for a different game thought thi...,1524056030628,0,1,neutral,settled for this game but it didnt meet my exp...,neutral
...,...,...,...,...,...,...,...,...,...
13997,5.0,My daughter loves this app!!,My daughter uses this app all the time and it ...,1623448427831,5,1,positive,my daughter loves this app my daughter uses th...,positive
13998,5.0,I’m not a fan but my daughter loves it!!,My daughter loves it!!,1623448062704,38,1,positive,i’m not a fan but my daughter loves it my daug...,positive
13999,5.0,Great,Great,1537991746835,0,1,positive,great great,positive
14000,5.0,Great radio,Love the 70s lite channel. Brings me back to m...,1471712622000,0,1,positive,great radio love the s lite channel brings me ...,positive


In [ ]:
df=origin[14000:50000]

In [ ]:
df

,rating,title,text,timestamp,helpful_vote,verified_purchase,label,full_text,sentiment
14002,5.0,Easter egg hunt for kids of all ages!! :),This game is a lot of fun. A fast yet fun way ...,1366146992000,1,1,positive,easter egg hunt for kids of all ages this game...,positive
14003,5.0,Great app. So far,"Great app. So far, any applications that's cal...",1504911259152,1,1,positive,great app so far great app so far any applicat...,positive
14004,5.0,Precision.,Regular.,1619791049855,0,1,positive,precision regular,neutral
14005,2.0,Tremenda porqueria,No me agrado...,1579310265560,1,1,negative,tremenda porqueria no me agrado,negative
14006,5.0,Calidad...,Me agrado...,1561855151237,0,1,positive,calidad me agrado,positive
...,...,...,...,...,...,...,...,...,...
50025,1.0,Can I give this 0 Stars?,"Seriously, I feel crazy for getting so angry a...",1551842273495,1,1,negative,can i give this stars seriously i feel crazy f...,negative
50026,5.0,Five Stars,Great gift cards for client. Everyone loves to...,1503635071176,0,1,positive,five stars great gift cards for client everyon...,positive
50027,4.0,It was good,The games good but after I updated it keeps po...,1465940361000,94,1,positive,it was good the games good but after i updated...,neutral
50028,3.0,its okay,It's just a city with a little bit of fun,1418952383000,0,1,neutral,its okay its just a city with a little bit of fun,neutral


In [ ]:
df.to_csv('/content/drive/MyDrive/sentiment_project/processed_reviews_third_badge.csv', index=False)

In [ ]:
df1 = pd.read_csv('/content/drive/MyDrive/sentiment_project/processed_reviews_7milles.csv')
df2 = pd.read_csv('/content/drive/MyDrive/sentiment_project/processed_reviews_second_7milles.csv')
df3 = pd.read_csv('/content/drive/MyDrive/sentiment_project/processed_reviews_third_badge.csv')


merged_df = pd.concat([df1, df2, df3], ignore_index=True)

In [ ]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   rating             50000 non-null  float64
 1   title              49972 non-null  object 
 2   text               49981 non-null  object 
 3   timestamp          50000 non-null  int64  
 4   helpful_vote       50000 non-null  int64  
 5   verified_purchase  50000 non-null  int64  
 6   label              50000 non-null  object 
 7   full_text          50000 non-null  object 
 8   sentiment          50000 non-null  object 
dtypes: float64(1), int64(3), object(5)
memory usage: 3.4+ MB


In [ ]:
merged_df

,rating,title,text,timestamp,helpful_vote,verified_purchase,label,full_text,sentiment
0,1.0,malware,mcaffee IS malware,1562182632076,0,0,negative,malware mcaffee is malware,negative
1,5.0,Lots of Fun,I love playing tapped out because it is fun to...,1424120336000,0,1,positive,lots of fun i love playing tapped out because ...,positive
2,5.0,Light Up The Dark,I love this flashlight app! It really illumin...,1362399267000,0,1,positive,light up the dark i love this flashlight app i...,positive
3,4.0,Fun game,One of my favorite games,1561061428662,0,1,positive,fun game one of my favorite games,positive
4,4.0,I am not that good at it but my kids are,Cute game. I am not that good at it but my kid...,1418257196000,0,1,positive,i am not that good at it but my kids are cute ...,positive
...,...,...,...,...,...,...,...,...,...
49995,1.0,Can I give this 0 Stars?,"Seriously, I feel crazy for getting so angry a...",1551842273495,1,1,negative,can i give this stars seriously i feel crazy f...,negative
49996,5.0,Five Stars,Great gift cards for client. Everyone loves to...,1503635071176,0,1,positive,five stars great gift cards for client everyon...,positive
49997,4.0,It was good,The games good but after I updated it keeps po...,1465940361000,94,1,positive,it was good the games good but after i updated...,neutral
49998,3.0,its okay,It's just a city with a little bit of fun,1418952383000,0,1,neutral,its okay its just a city with a little bit of fun,neutral


In [ ]:
# Save the merged DataFrame
merged_df.to_csv('/content/drive/MyDrive/sentiment_project/processed_reviews_50k.csv', index=False)

In [ ]:
merged_df['label_match'] = merged_df['label'] == merged_df['sentiment']

mismatches = merged_df[merged_df['label_match'] == False]

print(f"Number of mismatches: {len(mismatches)}")

pd.set_option('display.max_colwidth', None)
mismatches[['rating','full_text', 'label', 'sentiment']]

Number of mismatches: 6698


,rating,full_text,label,sentiment
13,2.0,its ok could be better for the amount of money the app is good but the interface leaves a lot to be desired i have it for roku so not sure if its the same on each platform when doing certain things like searching through your shows you have to go through everysingleoneofthem to get to the show you want instead of having each show in its own section like other apps also if you are searching through your shows and you click on it and then go back you go back to the very beginning when looking through recorded shows you scroll through a ton then you can hit see more and again it takes you back to the beginning the show selection is good and i am happy with that part of it but it gets old doing all the scrolling when they could improve the interface and make it a lot easier to see what you have to watch also you cannot always find shows in the search section it is odd and another one of frndlys weird features,negative,neutral
14,4.0,mobileme upgrade again cheaper to buy the amazon upgrade versus paining apples high pricebr to be honest i wonder at times why i do this every year the features really arent any better each year i dont have a lot backed up but do have somebr i havent hung a website up so whybr keeps an email available to me my wife and daughter to communicate with yes i have gmail and hotmail but both have been spammed and taken over even w password changesbr so i pay the fee and upgrade thanks to amazon,positive,negative
19,5.0,silly penguin lot of fun but can be very frustrating only because im a moron times while playing the gamemade me laugh myself and try harderjust a fun simple game,positive,neutral
24,4.0,reef rescue i like this game ok but wish there were less that you must purchase eh ill play until i get tired of the pay for this pay for that eh wish it could be the good old days and you could play a game for fun,positive,neutral
34,3.0,ugh super super tight buying tokens didnt help requires you to allow it access to your facebook if you want to register otherwise you play as an anonymous guest even if you buy tokens frankly i would pass unless you like selling your facebook ad space and selling out all your friends and family,neutral,negative
...,...,...,...,...
49924,5.0,untainted by the times as close to an unbiased look into world affairs with minimal liberal fact stretching,positive,negative
49928,5.0,not sure whats this is about not sure about this,positive,neutral
49932,2.0,beware advertisements works greatbr however even when not in use it sends advertisementso to my phone,negative,neutral
49987,3.0,eats batter fast norton has been a recognizeable name in computer security for years now theyve released a version of their software than can be used for the phone or tablet it work both on the ios and android devicesbr br after what seemed like an hour of scanning the qr code accessing the website installing the app setting up an account etc i finally had the app up and running heres the thing just like their desktop product the app is slow and a bit wonky theres not a specific problem with it but my device samsang galaxy s does go through the battery faster than it did before since i havent installed other apps im assuming that this app is seating up a lot of the energybr br norton will promote the fact that it is subscription and webbased as a good thing but i disagree id rather just buy the app once and have it for the life of my phone than to have to pay for it to renew it every months especially if i might switch out my phone after or monthsbr br the most useful feature is the remote locate option which does allow you to track and hopefully find your phone or tablet after its been lost or stolenbr br overall this product is okay but it has so much potential to be better i only hope that norton invests time money and resources into improving it by the time the subscription needs to be renewed,neutral,positive


In [4]:
dff=pd.read_csv('/content/drive/MyDrive/sentiment_project/processed_reviews_50k.csv')

In [8]:
dff = dff.dropna()


In [10]:
dff.info()

<class 'pandas.core.frame.DataFrame'>
Index: 49962 entries, 0 to 49999
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   rating             49962 non-null  float64
 1   title              49962 non-null  object 
 2   text               49962 non-null  object 
 3   timestamp          49962 non-null  int64  
 4   helpful_vote       49962 non-null  int64  
 5   verified_purchase  49962 non-null  int64  
 6   label              49962 non-null  object 
 7   full_text          49962 non-null  object 
 8   sentiment          49962 non-null  object 
dtypes: float64(1), int64(3), object(5)
memory usage: 3.8+ MB


In [11]:
dff.to_csv('/content/drive/MyDrive/sentiment_project/processed_reviews_50k.csv', index=False)